In [1]:
import numpy as np
import pandas as pd
from astroquery.jplhorizons import Horizons
from astropy.time import Time
from astropy import units as u
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt
import time
from astropy.coordinates import GeocentricMeanEcliptic
from astropy.coordinates import HeliocentricMeanEcliptic
from astropy.coordinates import HeliocentricTrueEcliptic
from astropy.coordinates import GeocentricTrueEcliptic
from astropy.coordinates import BarycentricTrueEcliptic
from astropy.coordinates import BarycentricMeanEcliptic
from astropy import coordinates 
import astropy

'''
    NAME:
         convert_to_primary_centric
         
    PURPOSE:
         This function takes a parameter Dataframe in RA/DEC, and converts it to Latitude 
         and Longitude, while also converting the dates to Primary-Centric Julian dates
         
    CALLING SEQUENCE:
         convert_to_primary_centric(paramsDF, objectName)
   
    INPUTS
          paramsDF - A dataframe of the observed positional data of the KBO in question
          objectName - The name of the object being observed (needed for the Horizons function)
   
    OUTPUTS:
          None. Just makes plots currently.
'''
def convert_to_primary_centric(paramsDF, objectNames, numobjects):
     #Current column names are just descriptive, not representative of final product column names
    updatedDF = pd.DataFrame(columns = ['time'])
    
    #Convert the dates into a Julian date format
    date = paramsDF['time']
    dateList = []
    for i in date:
        jd = Time(i,format='jd')
        dateList.append(jd)
        
    #Get the Horizons data for the object at the times it was observed
    primary = Horizons(id=objectNames[0],location=None,epochs=dateList)
    
    updatedDF['time'] = paramsDF['time']-primary.vectors()['lighttime']
    
    
    #Pull all data from csv file
    #RA_Prim = np.array(paramsDF['RA-Primary'])
    #DEC_Prim = np.array(paramsDF['DEC-Primary'])
    RA_Prim = np.array(primary.ephemerides()['RA'][:])
    DEC_Prim = np.array(primary.ephemerides()['DEC'][:])
    
    deltas = np.zeros(4*(len(objectNames)-1))
    for i in range(len(objectNames)-1):
    
        deltaRA_1 = np.array(paramsDF['Delta-RA_'+objectNames[i+1]]).astype(np.float)
        deltaDEC_1 = np.array(paramsDF['Delta-DEC_'+objectNames[i+1]]).astype(np.float)
    #deltaRA_Hiisi = np.array(paramsDF['Delta-RA_Hiisi'])
    #deltaDEC_Hiisi = np.array(paramsDF['Delta-DEC_Hiisi'])
    
        RA_1_err = np.array(paramsDF['Delta-RA_'+objectNames[i+1]+'-err']).astype(np.float)
        DEC_1_err = np.array(paramsDF['Delta-DEC_'+objectNames[i+1]+'-err']).astype(np.float)
        
        deltas[i*4] = deltaRA_1
        deltas[i*4+1] = deltaDEC_1
        deltas[i*4+2] = RA_1_err
        deltas[i*4+3] = DEC_1_err

        RA_1 = RA_Prim+deltaRA_1/3600/np.cos(DEC_Prim*u.degree)
        DEC_1 = DEC_Prim + deltaDEC_1/3600
    
        RA_1_err = RA_Prim+(deltaRA_1+RA_1_err)/3600/np.cos(DEC_Prim*u.degree)
        DEC_1_err = DEC_Prim + (deltaDEC_1+DEC_1_err)/3600
      
    #RA_Hiisi = RA_Prim+deltaRA_Hiisi/3600/np.cos(DEC_Prim*u.degree)
    #DEC_Hiisi = DEC_Prim + deltaDEC_Hiisi/3600
    
    #RA_Hiisi_err = RA_Prim+(deltaRA_Hiisi+RA_Hiisi_err)/3600/np.cos(DEC_Prim*u.degree)
    #DEC_Hiisi_err = DEC_Prim + (deltaDEC_Hiisi+DEC_Hiisi_err)/3600
    
        radf = pd.DataFrame()
        radf['RA-Prim'] = RA_Prim
        radf['DEC-Prim'] = DEC_Prim
        radf['RA-'+objectNames[i+1]] = RA_1
        radf['DEC-'+objectNames[i+1]] = DEC_1
    #radf['RA-Hiisi'] = RA_Hiisi
    #radf['DEC-Hiisi'] = DEC_Hiisi
    
    #print(radf)
    
    #Essentially we define where the object is in our RA/DEC coordinate system. ICRS is the system our coordinates are in.
        dist = primary.vectors()['range']
        firstC = SkyCoord(ra=RA_1*u.degree, dec=DEC_1*u.degree, frame='icrs',distance = dist,unit=(u.deg,u.deg))
    #HiisiC = SkyCoord(ra=RA_Hiisi*u.degree, dec=DEC_Hiisi*u.degree, frame='icrs',distance = dist,unit=(u.deg,u.deg))
    
        firstC_err = SkyCoord(ra=RA_1_err*u.degree, dec=DEC_1_err*u.degree, frame='icrs',distance = dist,unit=(u.deg,u.deg))
    #HiisiC_err = SkyCoord(ra=RA_Hiisi_err*u.degree, dec=DEC_Hiisi_err*u.degree, frame='icrs',distance = dist,unit=(u.deg,u.deg))
    
        primC = SkyCoord(ra=RA_Prim*u.degree, dec=DEC_Prim*u.degree, frame='icrs',distance = dist,unit=(u.deg,u.deg))
    
    #Transform your icrs frame to a J2000 latitude and longitude ecliptic frame
        firstEcl = firstC.transform_to(HeliocentricMeanEcliptic(equinox='J2000'))
    #HiisiEcl = HiisiC.transform_to(HeliocentricMeanEcliptic(equinox='J2000'))
    
        firstEcl_err = firstC_err.transform_to(HeliocentricMeanEcliptic(equinox='J2000'))
    #HiisiEcl_err = HiisiC_err.transform_to(HeliocentricMeanEcliptic(equinox='J2000'))
    
        primEcl = primC.transform_to(HeliocentricMeanEcliptic(equinox='J2000'))
    
        Lat_Prim = primEcl.lat.degree
        Long_Prim = primEcl.lon.degree
    
        Lat_1 = firstEcl.lat.degree
        Long_1 = firstEcl.lon.degree
        
    #Lat_Hiisi = HiisiEcl.lat.degree
    #Long_Hiisi = HiisiEcl.lon.degree
    
        Lat_1_err = firstEcl_err.lat.degree
        Long_1_err = firstEcl_err.lon.degree
    
    #Lat_Hiisi_err = HiisiEcl_err.lat.degree
    #Long_Hiisi_err = HiisiEcl_err.lon.degree
    
        DeltaLat_1 = (Lat_1-Lat_Prim)*3600
        DeltaLong_1 = (Long_1-Long_Prim)*np.cos(Lat_Prim*u.degree)*3600
    
    #DeltaLat_Hiisi = (Lat_Hiisi-Lat_Prim)*3600
    #DeltaLong_Hiisi = (Long_Hiisi-Long_Prim)*np.cos(Lat_Prim*u.degree)*3600
    
        Lat_1_err_arc = (Lat_1_err-Lat_1)*3600
        Long_1_err_arc = (Long_1_err-Long_1)*3600
    
    #Lat_Hiisi_err_arc = (Lat_Hiisi_err-Lat_Hiisi)*3600
    #Long_Hiisi_err_arc = (Long_Hiisi_err-Long_Hiisi)*3600
    
        if i == 0:
            updatedDF['Lat_Prim'] = Lat_Prim
            updatedDF['Long_Prim'] = Long_Prim
    
        updatedDF['DeltaLat_'+objectNames[i+1]] = DeltaLat_1
        updatedDF['DeltaLong_'+objectNames[i+1]] = DeltaLong_1
    #updatedDF['DeltaLat_Hiisi'] = DeltaLat_Hiisi
    #updatedDF['DeltaLong_Hiisi'] = DeltaLong_Hiisi
    
        updatedDF['DeltaLat_'+objectNames[i+1]+'_err'] = Lat_1_err_arc
        updatedDF['DeltaLong_'+objectNames[i+1]+'_err'] = Long_1_err_arc
    #updatedDF['DeltaLat_Hiisi_err'] = Lat_Hiisi_err_arc
    #updatedDF['DeltaLong_Hiisi_err'] = Long_Hiisi_err_arc
   
    print(updatedDF)
    
    updatedDF.to_csv('New'+objectNames[0]+'_LatLon.csv')
    

In [2]:
paramsDF = pd.read_csv("erisdata.csv")

convert_to_primary_centric(paramsDF,['Eris','Dysnomia'],2)

FileNotFoundError: [Errno 2] No such file or directory: 'erisdata.csv'

In [ ]:
#This function makes a scatter plot based on the data given (x,y) data points
def plot_scatter(array,dictionary, words):
    for i in words:
            x, y = array[dictionary[i]]
            plt.scatter(x, y, color = 'red')
            if dictionary[i]==2:
                plt.text(x-0.008,y+0.012,i,fontsize=9)
            else:
                plt.text(x+0.002,y+0.006,i,fontsize=9)
    plt.ylabel('Latitude')
    plt.xlabel('Longitude')
    plt.show()

In [8]:
primary = Horizons(id="eris",location=None,epochs=dateList)

NameError: name 'dateList' is not defined